In [14]:
import numpy as np
from scipy import io as sio
from matplotlib import pyplot as plt

In [15]:
import typhon as tp

In [16]:
Tb_simulated = tp.arts.xml.load("./ClearSky_1D_Tb.xml")
Tb_BosungObs = np.array([26.193, 23.754, 22.435, 18.510, 15.617, 12.566, 12.388, 11.472, 
                         107.09, 123.677, 147.536, 178.933, 216.787, 250.225, 273.193, 282.353, 
                         286.576, 288.430, 289.69, 290.449, 291.361, 291.436])

In [18]:
tp.retrieval.bmci.BMCI(Tb_simulated, )

TypeError: __init__() missing 3 required positional arguments: 'y', 'x', and 's_o'